# Graph-SLAM types and helper methods

This tuto introduces you to the variable types that we defined for our SLAM problem, and also to some methods that we pre-programmed for you.

## Overview

Our SLAM system consists of a single monocular camera, and some markers or tags from the AprilTag family, which we will consider as our landmarks.

Our **factor graph** will have *keyframes*, *landmarks*, and *factors* relating them.

Geometrically, each **keyframe** in the graph will be a **3D pose**.

Also, each **landmark** in the graph will be a **3D pose**.

**Factors** in the graph will be of three different types:

- **Motion factor**, from one keyframe to the next.
- **Landmark factor**, from one keyframe to one landmark.
- **Prior factor**, from one keyframe to some absolute value, or from one landmark to some absolute value.

Each factor will be associated with a particular **cost function**.

The **total cost** to be minimized by the solver will be the sum of the costs of each one of the factors.



## Variable types

### A note on 3D orientations in estimation and optimization

The orientation part of the pose deserves special attention. 
3D orientations do not form a linear space, but a Lie group. One can represent such rotations in different ways:
- Euler angles
- Rotation matrices
- Quaternions
- Axis-Angle
- Rotation vector, or Angle-vector

All have their pros and cons:
- Euler angles: 
    - :-)   intuitive, 3 parameters
    - :-(   non-continuous, singularities, 12 different conventions!
- Rotation matrices: 
    - :-)   easy to operate with
    - :-(   9 parameters, highly redundant
- Quaternions: 
    - :-)   fairly easy to operate
    - :-(   4 parameters, some redundancy
- Axis-Angle: 
    - :-)   intuitive
    - :-(   difficult to operate, 4 parameters, redundant
- Rotation vector, or Angle-vector: 
    - :-)   3 parameters, no singularities
    - :-(   difficult to operate

Since our solver IPOPT only understands variables in vector spaces and does not like redundant representations, we resort to the **rotation vector** representation.

We circumvent the stated difficulty of operation by converting the rotation vector to a rotation matrix every time we need it.

A rotation vector $w$ relates to its equivalent rotation matrix $R$ through the exponential map (or Rodrigues formula)
$$
R = \exp(w_\times) \\
w_\times = \log(R)
$$
where $w_\times$ is a skew-symmetric matrix build from the vector $w$, and $\exp,\log$ are the exponential map defined in SO(3).

For this tuto, it suffices to say that our variable to estimate is $w$, and that we can go back and forth to its matrix equivalent $R$ using the exponential map.

See the exponential map in action using Pinocchio for normal numeric variables:

In [1]:
import numpy as np
import pinocchio as pin

w = np.array([.1,.2,.3])

# go from vector to SO(3)
R = pin.exp(w)

print('rot vector w = ', w)
print('rot matrix R = \n', R)

# go from SO(3) to vector
u = pin.log(R)

print('rot vector u = ', u)

rot vector w =  [0.1 0.2 0.3]
rot matrix R = 
 [[ 0.9357548  -0.28316496  0.21019171]
 [ 0.30293271  0.95058062 -0.06803132]
 [-0.18054008  0.12733457  0.97529031]]
rot vector u =  [0.1 0.2 0.3]


And using Casadi for symbolic computations used by the optimizer:

In [2]:
import casadi
from pinocchio import casadi as cpin
from gslam_april_tools import *

# Create the casadi optimization problem
opti = casadi.Opti()

# The optimization variable is the angle-vector w and the associated rotation R=exp(w)
w = opti.variable(3)
R = exp3(w)
u = log3(R)

print('rot vector w = ', w)
print('rot matrix R = ', R)
print('rot vector u = ', u)


rot vector w =  opti0_x_1
rot matrix R =  exp3(opti0_x_1){0}
rot vector u =  log3(exp3(opti0_x_1){0}){0}




### The class `OptiVariablePose3`

This is a Pose3 object that can be used in Casadi for symbolic computations. Our keyframes and landmarks will be of this type.

The class basically contains:
- an **ID**
- a **position** : a symbolic 3-vector 
- an **orientation** : a symbolic 3-vector
- a method to recover the rotation matrix
- a constructor that accepts initial values for position and orientation

This is the prototype and constructor:


In [3]:


class OptiVariablePose3:
    def __init__(self, opti, id, position, anglevector):
        self.id             = id
        self.position       = opti.variable(3)
        self.anglevector    = opti.variable(3)
        self.R              = exp3(self.anglevector)
        opti.set_initial(self.position, position)
        opti.set_initial(self.anglevector, anglevector)

To create a keyframe, or a landmark, simply do:

In [4]:
# some necesary variables
opti        = casadi.Opti()
pos_example = np.array([1,2,3])
ori_example = np.array([4,5,6])

# create one keyframe!
kf_id    = 1
keyframe = OptiVariablePose3(opti, kf_id, pos_example, ori_example)

# create one landmark!
lmk_id   = 11
landmark = OptiVariablePose3(opti, lmk_id, pos_example, ori_example)

Keyframes and landmarks are stored in Dictionnaries, indexed by their ID. Insertion is trivial:

In [5]:
keyframes = dict()
landmarks = dict()

keyframes[kf_id]  = keyframe
landmarks[lmk_id] = landmark

So, having an ID, one can retrieve the KF or the lmk easily, so access is also trivial:

In [6]:
lmk = landmarks[lmk_id]
kf  = keyframes[kf_id]

Adding new keyframes (or landmarks) is easy, just remember to increment their ID each time:

In [7]:
# increment ID and create new keyframe:
kf_id += 1
keyframe = OptiVariablePose3(opti, kf_id, pos_example, ori_example)
keyframes[kf_id]  = keyframe

# increment ID and create new landmark:
lmk_id += 1
landmark = OptiVariablePose3(opti, lmk_id, pos_example, ori_example)
landmarks[lmk_id]  = landmark

You can now see all KFs and lmks in your SLAM problem:

In [8]:
# will show IDs used as keys and stored in objects -- they must match!
for kid in keyframes:
    keyframe = keyframes[kid]
    print('KF  # :', kid, keyframe.id)
for lid in landmarks:
    landmark = landmarks[lid]
    print('lmk # :', lid, landmark.id)

KF  # : 1 1
KF  # : 2 2
lmk # : 11 11
lmk # : 12 12


### The class `Factor`

Factors in our example are fairly simple. There are three types, but all can be stored in a single class:
- Prior factor: concerns a single KF or a single Lmk
- Motion factor: concerns two consecutive KFs
- Landmark factor: concerns one KF and one lmk

We define the class `Factor` as follows:

In [9]:
class Factor:
    def __init__(self, type, id, i, j, meas, sqrt_info):
        self.type = type
        self.id = id
        self.i = i
        self.j = j
        self.meas = meas
        self.sqrt_info = sqrt_info

The fields of `Factor` are explained as follows:
- `type`: string with values in `{'prior','motion','landmark'}`
- `id`: integer
- `i`: the ID of the first concerned object
- `j`: the ID of the second concerned object
- `meas`: the measurement
- `sqrt_info`: the square-root of the information matrix

So to create a factor, you need to know each of these fields.

Factors are stored in a dictionary, indexed by their own ID.

#### Prior factor

It constrains one keyframe:



In [10]:
factors = dict()

fac_id = 0
kf_id = 1
dummy = 0
meas = np.array([0,0,0,0,0,0])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors

factor = Factor('prior', fac_id, kf_id, dummy, meas, sqrt_info)

factors[fac_id] = factor


#### Motion factor

It relates two consecutive keyframes:

In [11]:
kf1_id = 1
kf2_id = 2
meas = np.array([1,2,3,  3,2,1])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors
fac_id += 1

factor = Factor('motion', fac_id, kf1_id, kf2_id, meas, sqrt_info)

factors[fac_id] = factor



#### Landmark factor

It relates a keyframe and a landmark measured from that keyframe:

In [12]:
fac_id += 1
kf_id = 1
lmk_id = lmk_id # this was already defined above
meas = np.array([1,2,3,  3,2,1])
sqrt_info = np.eye(6) / 1e-3 # 1mm, 1mrad, std deviation errors

factor = Factor('landmark', fac_id, kf_id, lmk_id, meas, sqrt_info)

factors[fac_id] = factor


Knowing what kfs or lmks are referred to by the factor is easy: check the `type` and recover the objects accordingly:

In [13]:
if factor.type == 'prior':
    keyframe = keyframes[factor.i]
    # do stuff with this KF
elif factor.type == 'motion':
    keyframe1 = keyframes[factor.i]
    keyframe2 = keyframes[factor.j]
    # do stuff with these KFs
elif factor.type == 'landmark':
    keyframe = keyframes[factor.i]
    landmark = landmarks[factor.j]
    # do stuff with these KF and lmk
else:
    print("Unrecognized factor type! Must be one in \{'prior, 'motion', landmark'\}")


We use this mechanism to compute the total cost to minimize:

In [14]:
# define dummy cost functions
def cost_constant_position (sqrt_info, keyframe1, keyframe2):
    return 10
def cost_landmark (measurement, sqrt_info, keyframe, landmark):
    return 20
def cost_keyframe_prior (measurement, sqrt_info, keyframe):
    return 30

# compute total cost
def computeTotalCost(factors, keyframes, landmarks):
    totalcost = 0
    for fid in factors:
        factor = factors[fid]
        i = factor.i
        j = factor.j
        measurement = factor.meas
        sqrt_info   = factor.sqrt_info
        if factor.type == "motion":
            totalcost += cost_constant_position (sqrt_info, keyframes[i], keyframes[j])
        elif factor.type == "landmark":
            totalcost += cost_landmark (measurement, sqrt_info, keyframes[i], landmarks[j])
        elif factor.type == "prior":
            totalcost += cost_keyframe_prior (measurement, sqrt_info, keyframes[i])
        else:
            print('Error in the factor type: type not known')
    return totalcost

totalcost = computeTotalCost(factors, keyframes, landmarks)

print('total cost = ', totalcost)

total cost =  60


where the cost of each type of factor is computed by a dedicated cost function (in the example above, the cost functions are defined as dummy functions returnning fixed dummy costs). 

We see these cost functions below.

## Cost functions

Each factor stores the necessary information for computing a cost function that compares the actual measurement against a prediction of that measurement made from the current states the factor is associated to.

### Cost per factor

The cost function is responsible for computing the cost of a given factor. 

Since a factor involves a measurement and some states, we begin by computing the expected value $e$ of the measurement, given the states:
$$
e = h(x_1, x_2)
$$
where $h()$ is a particular observation model, and $x_i$ are the states involved in the factor.

Then we compare our true measurement $y$ against the expected value $e$, obtaining what we call the prediction error $z$:
$$
z = y \ominus e = y \ominus h(x_1,x_2)
$$

**NOTE:** see that we compare with a generic operator $\ominus$ and not with a simple subtraction $-$. This allows us to compare magnitudes that do not belong to vector spaces, such as orientations in 3D. Read on to know how we deal with them!

This error $z$ is then weighted by the square root of the information matrix, $W$, and then taken the squared norm. This yields the cost $c$ of the factor:
$$
c = || W \cdot z ||^2
$$

Since factors are all of different nature and relate different states, it is necessary to index all variables correctly in formulas.
Overall, the cost we have computed for a given factor $k$ is the following:
$$
c_k = ||W_k(y_k\ominus h_k(x_{i_k},x_{j_k}))||^2
$$
where:
- $k$ is the factor index
- $i_k$ and $j_k$ are the indices of the two states this factor is pointing to
- $h_k$ is the particular observation model for this factor
- $y_k$ and $W_k$ are the measurement and sqrt-info-matrix of this factor

### Total cost

The total cost for $K$ factors is the sum of the costs of all factors
$$
c = \sum_{k=0}^K c_k = \sum_{k=0}^K ||W_k(y_k \ominus h_k(x_{i_k},x_{j_k}))||^2
$$
This is the cost to be minimized by the solver.



### Landmark observation cost function

We can observe the computation of one of such costs for our SLAM. 
We study the case of a landmark observation from a keyframe. 
We note the following facts:
1. the measurement $y$ is that obtained by the Apriltag detector, in the form of a translation vector and a rotation vector, stacked in a 6-vector.
1. one of the states $x_1$ is a particular keyframe, indexed by field `i` in the factor
1. the other state $x_2$ is a landmark, indexed by field `j` in the factor
1. the error $z$ is computed according to a chain of operations:
   1. the measurement is a 6-vector with translational and rotational parts
      $$
      y = \begin{bmatrix}T_y \\ w_y\end{bmatrix}
      $$
   1. The keyframe and landmark states are expressed in position and orientation:
      $$
      \textrm{keyframe: } x_i = \begin{bmatrix}T_i\\R_i\end{bmatrix} \\
      \textrm{landmark: } x_j = \begin{bmatrix}T_j\\R_j\end{bmatrix}
      $$
   1. the expected measurement $e=h(x_i,x_j)$ is a 3D composition of keyframe and landmark states. It is therefore a 3d pose with translation $T_e$ and rotation matrix $R_e$
      $$
      e = \begin{bmatrix}T_e \\ R_e\end{bmatrix}
      $$
      this is computed as follows:
      $$
      T_e = R_i^\top (T_j - T_i) \\
      R_e = R_i^\top R_j
      $$
      
   1. the error is the "difference" of the predicted above with the measurement, expressed by $\ominus$
      $$
      z = y \ominus e
      $$
      1. for the translational part, the difference is a mere subtraction
         $$
         T_z = T_y - T_e
         $$
      1. for the rotational part, this "difference" is done in the SO3 manifold, with,
         $$z_w = \log(R_e^\top \cdot \exp(w_y))$$
      so that we have the error:
      $$
      z = y \ominus e = \begin{bmatrix} T_y - T_e \\ \log(R_e^\top \cdot \exp(w_y)) \end{bmatrix}
      $$

It seems complicated. A glipmse at the code may clarify things up:

In [15]:
# landmark observations
def cost_landmark(meas, sqrt_info, keyframe_i, landmark_j):
    # predict: landmark pose wrt KF pose
    ppred = keyframe_i.R.T @ (landmark_j.position - keyframe_i.position)
    Rpred = keyframe_i.R.T @ landmark_j.R

    # error: use manifold tools for the orientation part
    perr = meas[0:3] - ppred
    Rerr = Rpred.T @ exp3(meas[3:6])
    werr = log3(Rerr)

    err = casadi.vertcat(perr, werr)
 
    # apply weight and compute squared norm
    cost = casadi.sumsqr(sqrt_info @ err) 
    return cost


### Motion cost function: costant position model

Since we have no motion sensor, we assume the motion is null at each step. This is known as a constant-position motion model. What we do is:
- predict the motion: compose $x_1$ and $x_2$ so that you obtain a prediction that corresponds to the frame `keyframe2` with respect to `keyframe1`.
- compare this prediction with the null motion, that is, $y = 0_{6\times1}$

Since $y=0$, we can write the function prototype for the constant-position motion model with no measurement input. To allow for real motion, we have to provide an information matrix that is small.

Now, define a similar cost function for the motion, that is, between two keyframes.



In [16]:
def cost_constant_position(sqrt_info, keyframe_i, keyframe_j):
    # write y9our code here and change the return line below
    return 0  

### Prior keyframe cost function

Now, write a cost function to apply a prior pose to one keyframe:

In [17]:
def cost_keyframe_prior(meas, sqrt_info, keyframe_i):
    return 0

### Prior landmark cost function

Finally, write a cost function to apply a prior pose to a landmark:

In [18]:
def cost_landmark_prior(meas, sqrt_info, landmark):
    return 0

**NOTE:** you can have a prior in the first keyframe, or a prior in one of the landmarks. You CANNOT impose both priors, since they would most certainly conflict with each other!